In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


# 1)getting the data

In [2]:
import csv
filename='df.csv'
#Read csv
df = pd.read_csv(filename)
df.head(5)

,latitude,longitude,community
0,25.175900,51.526697,Al Bidda
1,25.192600,51.315000,Al Dafna
2,25.163300,51.321000,Ad Dawhah al Jadida
3,25.389002,51.509524,Al Egla
4,25.171200,51.320000,Al Hilal


The data has the name of the neighborhoods in Doha,Qatar and their locations.

## map of Doha 

In [3]:
map_doha = folium.Map(location=[25.1712,51.32], zoom_start=11)

for lat, lng, label in zip(df['latitude'],df['longitude'], df['community']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_doha)  
    
map_doha 

# 2) getting foursquare credentials 

In [4]:
CLIENT_ID = 'CSTJMFI5FTSRO3RJ11MO40FHQPURY0WS2XHIABMMMSNVO5VG' 
CLIENT_SECRET = 'B5HL0VXZJWCAR2R23AWDBO3EUYFZHECZWHOHZJCR0VBEFYR1' 
VERSION = '20180605'
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CSTJMFI5FTSRO3RJ11MO40FHQPURY0WS2XHIABMMMSNVO5VG
CLIENT_SECRET:B5HL0VXZJWCAR2R23AWDBO3EUYFZHECZWHOHZJCR0VBEFYR1


# 3) getting nearby venues for all communities

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'venue id',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
nearby_venues = getNearbyVenues(names=df['community'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'])


Al Bidda
Al Dafna
Ad Dawhah al Jadida
Al Egla
Al Hilal
Al Jasrah
Al Kharayej
Al Khulaifat
Al Mansour
Al Markhiya
Al Messila
Al Mirqab
Al Najada
Al Qassar
Al Rufaa
Al Sadd
Al Souq
Al Tarfa
Al Thumama
Barahat Al Jufairi
Dahl Al Hamam
Doha International Airport
Doha Por
Duhail
Hamad Medical City
Hazm Al Markhiya
Industrial Area
Jabal Thuaileb
Jelaiah
Jeryan Nejaima
Lejbailat
Lekhwair
Leqtaifiya
Madinat Khalifa North
Madinat Khalifa South
Musheireb
Najma
New Al Hitmi
New Al Mirqab
New Salata
Old Airport
Old Al Ghanim
Onaiza
Ras Abu Aboud
Rawdat Al Khail
Rumeilah
Salata
Umm Ghuwailina
Umm Lekhba
Wadi Al Banat
Wadi Al Sail


In [7]:
nearby_venues.shape

(262, 8)

In [8]:
nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,venue id,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Al Bidda,1,1,1,1,1,1,1
Al Egla,3,3,3,3,3,3,3
Al Mansour,19,19,19,19,19,19,19
Al Mirqab,33,33,33,33,33,33,33
Al Qassar,35,35,35,35,35,35,35
Al Rufaa,44,44,44,44,44,44,44
Al Tarfa,3,3,3,3,3,3,3
Dahl Al Hamam,16,16,16,16,16,16,16
Doha Por,9,9,9,9,9,9,9


# 4) analysis

In [9]:
doha_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
doha_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [doha_onehot.columns[-1]] + list(doha_onehot.columns[:-1])
doha_onehot = doha_onehot[fixed_columns]

doha_onehot.head()

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bookstore,Border Crossing,Brazilian Restaurant,Building,Burger Joint,Cafeteria,Café,Campground,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Jazz Club,Lounge,Market,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Museum,Nightclub,North Indian Restaurant,Optical Shop,Pakistani Restaurant,Palace,Park,Pastry Shop,Persian Restaurant,Peruvian Restaurant,Pizza Place,Playground,Pool,Pool Hall,Pub,Rental Car Location,Resort,Restaurant,Rugby Pitch,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Spa,Sri Lankan Restaurant,Supermarket,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Al Bidda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Al Egla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Al Egla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Al Egla,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Al Mansour,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [10]:
doha_grouped = doha_onehot.groupby('Neighborhood').mean().reset_index()
doha_grouped

,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bookstore,Border Crossing,Brazilian Restaurant,Building,Burger Joint,Cafeteria,Café,Campground,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Jazz Club,Lounge,Market,Medical Center,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Museum,Nightclub,North Indian Restaurant,Optical Shop,Pakistani Restaurant,Palace,Park,Pastry Shop,Persian Restaurant,Peruvian Restaurant,Pizza Place,Playground,Pool,Pool Hall,Pub,Rental Car Location,Resort,Restaurant,Rugby Pitch,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Spa,Sri Lankan Restaurant,Supermarket,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Al Bidda,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Al Egla,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.333333,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Al Mansour,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.052632,0.000000,0.000000,0.052632,0.052632,0.0,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.052632,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.052632,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.052632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.052632,0.052632,0.0000,0.052632,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000
3,Al Mirqab,0.000000,0.0,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000

In [11]:
doha_grouped.shape

(21, 99)

### print each neighborhood along with the top 5 most common venues

In [12]:
num_top_venues = 5

for hood in doha_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = doha_grouped[doha_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Al Bidda----
                  venue  freq
0     College Cafeteria   1.0
1   American Restaurant   0.0
2           Pastry Shop   0.0
3                Palace   0.0
4  Pakistani Restaurant   0.0


----Al Egla----
                 venue  freq
0          Rugby Pitch  0.33
1                  Gym  0.33
2               Lounge  0.33
3  American Restaurant  0.00
4               Palace  0.00


----Al Mansour----
                  venue  freq
0     Indian Restaurant  0.11
1  Fast Food Restaurant  0.11
2                   Spa  0.05
3     Electronics Store  0.05
4    Falafel Restaurant  0.05


----Al Mirqab----
        venue  freq
0       Hotel  0.42
1        Café  0.09
2  Restaurant  0.06
3      Museum  0.06
4         Pub  0.06


----Al Qassar----
        venue  freq
0  Restaurant  0.11
1       Beach  0.11
2      Resort  0.06
3   Nightclub  0.06
4   Hotel Bar  0.06


----Al Rufaa----
                  venue  freq
0                 Hotel  0.32
1                  Café  0.09
2             BBQ Joi

###  create new dataframe and display the top 10 venues for each neighborhood.

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = doha_grouped['Neighborhood']

for ind in np.arange(doha_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(doha_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Bidda,College Cafeteria,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner,Electronics Store
1,Al Egla,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
2,Al Mansour,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
3,Al Mirqab,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
4,Al Qassar,Restaurant,Beach,Nightclub,Resort,Hotel Bar,American Restaurant,Mediterranean Restaurant,Chinese Restaurant,Coffee Shop,Middle Eastern Restaurant


# 5) clustring

In [15]:
# set number of clusters
kclusters =10

doha_grouped_clustering = doha_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(doha_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 7, 1, 8, 1, 8, 6, 1, 1, 9], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

doha_merged = nearby_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
doha_merged = doha_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

doha_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,venue id,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Al Bidda,25.175900,51.526697,Food Court - Qatar University,4f719f0ae4b030300c798bf5,25.179399,51.526489,College Cafeteria,3,College Cafeteria,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner,Electronics Store
1,Al Egla,25.389002,51.509524,The Twenty Four,54bba465498ec3ed3da8f6b3,25.387820,51.507952,Lounge,7,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
2,Al Egla,25.389002,51.509524,Strongbox,5cf531ae3fcee8002c55491a,25.388650,51.506569,Gym,7,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
3,Al Egla,25.389002,51.509524,Doha Sports Park,5dd160c156b2c80008078f15,25.387924,51.506763,Rugby Pitch,7,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
4,Al Mansour,25.269300,51.529500,حلويات النشامى,500af634e4b0fec2cd843dd1,25.272191,51.526486,Mediterranean Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach


# 6)map

In [17]:
# create map
map_clusters = folium.Map(location=[25.1712,51.32], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(doha_merged['Venue Latitude'], doha_merged['Venue Longitude'], doha_merged['Neighborhood'], doha_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 7) analyzing clusters

cluster 1

In [18]:
doha_merged.loc[doha_merged['Cluster Label'] == 0, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
223,25.298,25.297384,51.496908,Lounge,0,Café,Gym,Restaurant,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop
224,25.298,25.301884,51.492683,Café,0,Café,Gym,Restaurant,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop
225,25.298,25.301496,51.490380,Café,0,Café,Gym,Restaurant,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop
226,25.298,25.301170,51.495589,Gym,0,Café,Gym,Restaurant,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop
227,25.298,25.302151,51.492934,Restaurant,0,Café,Gym,Restaurant,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop


cluster 2

In [19]:
doha_merged.loc[doha_merged['Cluster Label'] == 1, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,25.269300,25.272191,51.526486,Mediterranean Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
5,25.269300,25.273369,51.530212,Beach,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
6,25.269300,25.271823,51.528983,Fast Food Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
7,25.269300,25.271880,51.533264,Pakistani Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
8,25.269300,25.273069,51.527533,Pool Hall,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
9,25.269300,25.271773,51.528959,Supermarket,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
10,25.269300,25.272054,51.526474,Snack Place,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
11,25.269300,25.268878,51.532959,Turkish Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
12,25.269300,25.271857,51.528983,Fast Food Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach
13,25.269300,25.271793,51.529017,Indian Restaurant,1,Indian Restaurant,Fast Food Restaurant,Pakistani Restaurant,Asian Restaurant,Mediterranean Restaurant,Department Store,Electronics Store,Falafel Restaurant,Pool Hall,Beach


cluster 3

In [20]:
doha_merged.loc[doha_merged['Cluster Label'] == 2, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
228,25.16,25.15859,51.305812,Farm,2,Farm,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
239,25.16,25.15859,51.305812,Farm,2,Farm,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner


cluster 4

In [21]:
doha_merged.loc[doha_merged['Cluster Label'] == 3, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,25.1759,25.179399,51.526489,College Cafeteria,3,College Cafeteria,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner,Electronics Store


cluster 5 

In [22]:
doha_merged.loc[doha_merged['Cluster Label'] == 4, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
171,25.374,25.377263,51.476117,Castle,4,Castle,Chinese Restaurant,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner,Electronics Store


cluster 6

In [23]:
doha_merged.loc[doha_merged['Cluster Label'] == 5, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
195,25.329167,25.331198,51.476921,Grocery Store,5,Medical Center,Grocery Store,Farm,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
196,25.329167,25.332507,51.478434,Medical Center,5,Medical Center,Grocery Store,Farm,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner


cluster 7

In [24]:
doha_merged.loc[doha_merged['Cluster Label'] == 6, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
135,25.3755,25.377052,51.489117,Coffee Shop,6,Food Court,Coffee Shop,Vegetarian / Vegan Restaurant,Farm,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
136,25.3755,25.373645,51.487381,Food Court,6,Food Court,Coffee Shop,Vegetarian / Vegan Restaurant,Farm,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
137,25.3755,25.373754,51.487503,Food Court,6,Food Court,Coffee Shop,Vegetarian / Vegan Restaurant,Farm,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner


cluster 8

In [25]:
doha_merged.loc[doha_merged['Cluster Label'] ==7, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,25.389002,25.387820,51.507952,Lounge,7,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
2,25.389002,25.388650,51.506569,Gym,7,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner
3,25.389002,25.387924,51.506763,Rugby Pitch,7,Gym,Rugby Pitch,Lounge,Coffee Shop,College Cafeteria,Convenience Store,Currency Exchange,Department Store,Dessert Shop,Diner


cluster 9

In [26]:
doha_merged.loc[doha_merged['Cluster Label'] == 8, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,25.288642,25.288565,51.544506,Hotel,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
24,25.288642,25.290216,51.543960,Hotel,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
25,25.288642,25.288298,51.544541,Restaurant,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
26,25.288642,25.289456,51.543685,Hotel,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
27,25.288642,25.287249,51.542147,Furniture / Home Store,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
28,25.288642,25.290198,51.544538,Restaurant,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
29,25.288642,25.291061,51.537760,Museum,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
30,25.288642,25.291002,51.539006,Museum,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
31,25.288642,25.288750,51.544524,Lounge,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge
32,25.288642,25.290164,51.542482,Motel,8,Hotel,Café,Restaurant,Museum,Pub,Furniture / Home Store,Cocktail Bar,Motel,Miscellaneous Shop,Lounge


cluster 10

In [27]:
doha_merged.loc[doha_merged['Cluster Label'] == 9, doha_merged.columns[[1] + list(range(5, doha_merged.shape[1]))]]

,Neighborhood Latitude,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
163,25.3491,25.347779,51.470050,Gym / Fitness Center,9,Gym / Fitness Center,Candy Store,Campground,Pool,Vegetarian / Vegan Restaurant,Cocktail Bar,College Cafeteria,Convenience Store,Currency Exchange,Department Store
164,25.3491,25.350449,51.468314,Pool,9,Gym / Fitness Center,Candy Store,Campground,Pool,Vegetarian / Vegan Restaurant,Cocktail Bar,College Cafeteria,Convenience Store,Currency Exchange,Department Store
165,25.3491,25.348388,51.472185,Candy Store,9,Gym / Fitness Center,Candy Store,Campground,Pool,Vegetarian / Vegan Restaurant,Cocktail Bar,College Cafeteria,Convenience Store,Currency Exchange,Department Store
166,25.3491,25.345602,51.470171,Campground,9,Gym / Fitness Center,Candy Store,Campground,Pool,Vegetarian / Vegan Restaurant,Cocktail Bar,College Cafeteria,Convenience Store,Currency Exchange,Department Store
